In [ ]:
!pip install flask_ngrok

In [ ]:
from flask import Flask, render_template, request
import pickle
import numpy as np
from flask_ngrok import run_with_ngrok
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
import xgboost as xgb
import pandas as pd
import joblib
import re

In [ ]:
cd /content/drive/MyDrive/Case_study mercedes_Manufacturing/deployment

/content/drive/MyDrive/Case_study mercedes_Manufacturing/deployment


In [ ]:
with open('encoded/le_dict_X0.pickle', 'rb') as handle:
    X0_dict = pickle.load(handle)
with open('encoded/le_dict_X1.pickle', 'rb') as handle:
    X1_dict = pickle.load(handle)
with open('encoded/le_dict_X2.pickle', 'rb') as handle:
    X2_dict = pickle.load(handle)
with open('encoded/le_dict_X3.pickle', 'rb') as handle:
    X3_dict = pickle.load(handle)
with open('encoded/le_dict_X5.pickle', 'rb') as handle:
    X5_dict = pickle.load(handle)
with open('encoded/le_dict_X6.pickle', 'rb') as handle:
    X6_dict = pickle.load(handle)
with open('encoded/le_dict_X8.pickle', 'rb') as handle:
    X8_dict = pickle.load(handle)            
dict_cat_feature=[X0_dict , X1_dict , X2_dict , X3_dict , X5_dict , X6_dict , X8_dict]

In [ ]:
def preprocess_data(train):
  '''
  in preprocessing function i am label encoding 
  categorical function
  '''
  with open('features.pkl', 'rb') as f:
   final_label = pickle.load(f)
   final_label.remove("y")
  df_train=train[final_label]
  cat_feat=["X0" ,"X1" ,	"X2" ,	"X3" ,	"X5" ,	"X6" ,	"X8"] #categorical function
  for i  , j in enumerate(cat_feat):
    df_train[j]   = label_encoding_binary(df_train ,j ,dict_cat_feature[i])
  return df_train

######################################################################
def input_list(X):
  '''
  converting string list into int list
  '''
  final_list=[]
  for i in X[0].split(","):
    if i.strip().isdigit():
      final_list.append(int(i))
    else: final_list.append(i.strip())
  return final_list

################################################################################
def df(test):
  ''' 
  in given function i am converting list into data frame
  '''
  with open('train_col.pkl', 'rb') as f:
    train_col = pickle.load(f)
  df=pd.DataFrame(test ,train_col).T
  return df
################################################################################
def label_encoding_binary(df_train  , feature , le_dict):
  '''
  label encoding each features
  '''
  labelenc=[]
  for i in df_train[feature]:
    if i in le_dict:labelenc.append(le_dict[i])
    else:labelenc.append(0)
  return  labelenc  
################################################################################
def final1(X):
  ''' 
  in given function i am taking list as input , 
  converting list into data frame 
  then label encoding categorical features , 
  then i am using model 1 and model 2 to predict target variable and  
  average both prediction
  '''
  with open('train_col.pkl', 'rb') as f:
    train_col = pickle.load(f)
  df=pd.DataFrame(X ,train_col).T
  X_processed=preprocess_data(df)
  
  model1= joblib.load("avg_model_XGB")
  model2= joblib.load("avg_model_2")
  train_original_np=X_processed.to_numpy(dtype ='float32')

  y_pred1 = model1.predict(train_original_np)

  dtest = xgb.DMatrix(train_original_np)
  y_pred2 = model2.predict(dtest)

  avg_pred = (y_pred1+y_pred2)/2
  return avg_pred

In [ ]:
test1=[1 ,'az', 'v' ,'n', 'f' ,'d', 't', 'a', 'w', 0, 0, 0, 0, 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0 ,0 ,0, 0, 1, 1, 1,
 0 ,1, 0, 0, 0, 1, 0, 1, 0, 0 ,0 ,0, 0 ,1, 0, 0, 1, 0, 0, 0, 0, 0,0 ,0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0 ,0, 0, 0,
 0, 0 ,0, 1, 0, 0, 1 ,0, 1, 0, 0, 0, 1, 0 ,0, 0, 0 ,0, 0 ,0 ,0, 0, 0 ,0 ,0, 0, 0 ,0, 1 ,0 ,1 ,0 ,0, 1 ,0, 0, 0,
 0 ,0, 0 ,0 ,0, 0, 1 ,0 ,0, 1, 0 ,0 ,0, 0, 0, 0, 0, 0, 0, 0 ,0, 0, 1, 0, 0, 0, 1 ,0 ,0, 0, 0, 0 ,0 ,0, 0, 0, 0,
 0, 1, 0, 0 ,0 ,1 ,1 ,0 ,0 ,0 ,0 ,0, 0, 1 ,1, 0, 0, 0, 1, 0, 0 ,0 ,1, 0, 0, 0 ,0 ,0 ,0, 0 ,0, 0, 0, 0, 0,1, 0,
 0 ,0 ,0 ,0, 1, 0, 0 ,0, 0 ,0 ,0 ,1, 0 ,0 ,0 , 0, 0, 0, 0, 0, 1, 0, 0, 0 ,0 ,1 ,0, 0, 0 ,0, 0, 0,0 ,0 ,1 ,0,
 1 ,0, 0 ,1 ,0 ,0 ,0 ,0 ,0, 0, 0 ,0 ,1, 0, 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0 ,0 ,0, 0 ,1 ,0 ,0 ,0 ,1, 0, 0 ,0 ,0, 0, 1,
 0 ,0 ,0 ,0 ,0, 0, 0, 0 ,0 ,0, 0 ,0 ,0 ,0 ,0, 1 ,1 ,0 ,0, 1, 0, 0 ,1, 0 ,0, 0, 0, 0, 0, 1, 0, 0, 0 ,0 ,0, 0, 0,
 0 ,0 ,0, 0 ,0 ,0, 0 ,0, 0, 1 ,0 ,0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0 ,0 ,0, 0 ,0, 0, 0, 0, 0, 1 ,0, 0, 0,
 0 ,0 ,1, 0, 0 ,0, 0 ,0, 0, 0, 0 ,0, 0, 0 ,0 ,0, 1, 0, 1 ,0, 0, 0, 0 ,0 ,0 ,0, 1, 0 ,0, 1, 0, 1 ,0, 0 ,0 ,0, 0,
 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0, 1, 0 ,0, 0, 0, 0, 0,0]
final1(test1)

[03:18:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([86.50781], dtype=float32)

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)
######################################################################
@app.route('/')
def hello_world():
    return 'Hello World!'


@app.route('/index')
def index():
    return flask.render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
  to_predict_list = request.form.getlist("feature")
  to_predict_list= [n.strip() for n in to_predict_list]
  final_list=input_list(to_predict_list)
  final_score=final1(final_list)
  return str(final_score)
######################################################################
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c85f-34-86-189-104.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Feb/2022 17:51:34] "GET / HTTP/1.1" 200 -


In [ ]:
test1=[1 ,'az', 'v' ,'n', 'f' ,'d', 't', 'a', 'w', 0, 0, 0, 0, 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0 ,0 ,0, 0, 1, 1, 1,
 0 ,1, 0, 0, 0, 1, 0, 1, 0, 0 ,0 ,0, 0 ,1, 0, 0, 1, 0, 0, 0, 0, 0,0 ,0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0 ,0, 0, 0,
 0, 0 ,0, 1, 0, 0, 1 ,0, 1, 0, 0, 0, 1, 0 ,0, 0, 0 ,0, 0 ,0 ,0, 0, 0 ,0 ,0, 0, 0 ,0, 1 ,0 ,1 ,0 ,0, 1 ,0, 0, 0,
 0 ,0, 0 ,0 ,0, 0, 1 ,0 ,0, 1, 0 ,0 ,0, 0, 0, 0, 0, 0, 0, 0 ,0, 0, 1, 0, 0, 0, 1 ,0 ,0, 0, 0, 0 ,0 ,0, 0, 0, 0,
 0, 1, 0, 0 ,0 ,1 ,1 ,0 ,0 ,0 ,0 ,0, 0, 1 ,1, 0, 0, 0, 1, 0, 0 ,0 ,1, 0, 0, 0 ,0 ,0 ,0, 0 ,0, 0, 0, 0, 0,1, 0,
 0 ,0 ,0 ,0, 1, 0, 0 ,0, 0 ,0 ,0 ,1, 0 ,0 ,0 , 0, 0, 0, 0, 0, 1, 0, 0, 0 ,0 ,1 ,0, 0, 0 ,0, 0, 0,0 ,0 ,1 ,0,
 1 ,0, 0 ,1 ,0 ,0 ,0 ,0 ,0, 0, 0 ,0 ,1, 0, 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0 ,0 ,0, 0 ,1 ,0 ,0 ,0 ,1, 0, 0 ,0 ,0, 0, 1,
 0 ,0 ,0 ,0 ,0, 0, 0, 0 ,0 ,0, 0 ,0 ,0 ,0 ,0, 1 ,1 ,0 ,0, 1, 0, 0 ,1, 0 ,0, 0, 0, 0, 0, 1, 0, 0, 0 ,0 ,0, 0, 0,
 0 ,0 ,0, 0 ,0 ,0, 0 ,0, 0, 1 ,0 ,0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0 ,0 ,0, 0 ,0, 0, 0, 0, 0, 1 ,0, 0, 0,
 0 ,0 ,1, 0, 0 ,0, 0 ,0, 0, 0, 0 ,0, 0, 0 ,0 ,0, 1, 0, 1 ,0, 0, 0, 0 ,0 ,0 ,0, 1, 0 ,0, 1, 0, 1 ,0, 0 ,0 ,0, 0,
 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0, 1, 0 ,0, 0, 0, 0, 0,0]

In [ ]:
test2=[7, 'az', 'w', 'n', 'c', 'd', 'x', 'j', 'x', 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]